# Segmenting and Clustering Neighborhoods in the US

###### Perpared by Enrique Puente for Coursera's Applied Data Science Capston Project

The United States has  a housing problem for some time now. Housing prices keep increasing as restrictions hinder  development in the main cities. Meanwhile suburban areas are deemed unattractive as there's "not enough going on" there. One view of this notion could be the lack of public venues for people to enjoy. The thesis is that there must be neighborhoods in the United States that may qualify for home pursuers criteria and may not necessarily be in such densely populated areas. This study intends to analyze areas in the US and describe them by means of the venues around those areas and relative pricing. By the end of this analysis we should have uncovered the best alternatives for an audience that wants to relocate after the their job allows for remote working.

## 1.0 Download and Import Libraries

In [2]:
# Import Librarires
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from geopy.exc import GeocoderTimedOut
import time

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
# !conda install -c conda-forge saeborn --yes
# import saeborn

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 2.0 Data Download

### 2.1 Loading Data from CSV (Zillow Provided)

The following cell uses Pandas' "read_csv" method for pulling table from CSV file. The file contains a list of nighborhoods associated with cities and states across the US. The file also contains the Zillow Home Value Index (ZHVI), a smoothed, seasonally adjusted measure of the typical home value and market changes across a given region and housing type. According to Zillow. Because the stock of homes was fixed over time, month-over-month growth under these assumptions could be interpreted as appreciation of the typical home.
For more infromation access Zillow's methodology secription at the following link: https://www.zillow.com/research/zhvi-methodology-2019-highlights-26221/

In [3]:
# Load CSV file
Zdf = pd.read_csv('Neighborhood_Zhvi_AllHomes.csv')
Zdf.index = Zdf['RegionID']
Addressdf = Zdf[['RegionName', 'City', 'State']].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)
Zdf.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1/31/96,2/29/96,3/31/96,4/30/96,5/31/96,6/30/96,7/31/96,8/31/96,9/30/96,10/31/96,11/30/96,12/31/96,1/31/97,2/28/97,3/31/97,4/30/97,5/31/97,6/30/97,7/31/97,8/31/97,9/30/97,10/31/97,11/30/97,12/31/97,1/31/98,2/28/98,3/31/98,4/30/98,5/31/98,6/30/98,7/31/98,8/31/98,9/30/98,10/31/98,11/30/98,12/31/98,1/31/99,2/28/99,3/31/99,4/30/99,5/31/99,6/30/99,7/31/99,8/31/99,9/30/99,10/31/99,11/30/99,12/31/99,1/31/00,2/29/00,3/31/00,4/30/00,5/31/00,6/30/00,7/31/00,8/31/00,9/30/00,10/31/00,11/30/00,12/31/00,1/31/01,2/28/01,3/31/01,4/30/01,5/31/01,6/30/01,7/31/01,8/31/01,9/30/01,10/31/01,11/30/01,12/31/01,1/31/02,2/28/02,3/31/02,4/30/02,5/31/02,6/30/02,7/31/02,8/31/02,9/30/02,10/31/02,11/30/02,12/31/02,1/31/03,2/28/03,3/31/03,4/30/03,5/31/03,6/30/03,7/31/03,8/31/03,9/30/03,10/31/03,11/30/03,12/31/03,1/31/04,2/29/04,3/31/04,4/30/04,5/31/04,6/30/04,7/31/04,8/31/04,9/30/04,10/31/04,11/30/04,12/31/04,1/31/05,2/28/05,3/31/05,4/30/05,5/31/05,6/30/05,7/31/05,8/31/05,9/30/05,10/31/05,11/30/05,12/31/05,1/31/06,2/28/06,3/31/06,4/30/06,5/31/06,6/30/06,7/31/06,8/31/06,9/30/06,10/31/06,11/30/06,12/31/06,1/31/07,2/28/07,3/31/07,4/30/07,5/31/07,6/30/07,7/31/07,8/31/07,9/30/07,10/31/07,11/30/07,12/31/07,1/31/08,2/29/08,3/31/08,4/30/08,5/31/08,6/30/08,7/31/08,8/31/08,9/30/08,10/31/08,11/30/08,12/31/08,1/31/09,2/28/09,3/31/09,4/30/09,5/31/09,6/30/09,7/31/09,8/31/09,9/30/09,10/31/09,11/30/09,12/31/09,1/31/10,2/28/10,3/31/10,4/30/10,5/31/10,6/30/10,7/31/10,8/31/10,9/30/10,10/31/10,11/30/10,12/31/10,1/31/11,2/28/11,3/31/11,4/30/11,5/31/11,6/30/11,7/31/11,8/31/11,9/30/11,10/31/11,11/30/11,12/31/11,1/31/12,2/29/12,3/31/12,4/30/12,5/31/12,6/30/12,7/31/12,8/31/12,9/30/12,10/31/12,11/30/12,12/31/12,1/31/13,2/28/13,3/31/13,4/30/13,5/31/13,6/30/13,7/31/13,8/31/13,9/30/13,10/31/13,11/30/13,12/31/13,1/31/14,2/28/14,3/31/14,4/30/14,5/31/14,6/30/14,7/31/14,8/31/14,9/30/14,10/31/14,11/30/14,12/31/14,1/31/15,2/28/15,3/31/15,4/30/15,5/31/15,6/30/15,7/31/15,8/31/15,9/30/15,10/31/15,11/30/15,12/31/15,1/31/16,2/29/16,3/31/16,4/30/16,5/31/16,6/30/16,7/31/16,8/31/16,9/30/16,10/31/16,11/30/16,12/31/16,1/31/17,2/28/17,3/31/17,4/30/17,5/31/17,6/30/17,7/31/17,8/31/17,9/30/17,10/31/17,11/30/17,12/31/17,1/31/18,2/28/18,3/31/18,4/30/18,5/31/18,6/30/18,7/31/18,8/31/18,9/30/18,10/31/18,11/30/18,12/31/18,1/31/19,2/28/19,3/31/19,4/30/19,5/31/19,6/30/19,7/31/19,8/31/19,9/30/19,10/31/19,11/30/19,12/31/19,1/31/20,2/29/20,3/31/20,4/30/20
RegionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
274772,274772,0,Northeast Dallas,Neighborhood,TX,TX,Dallas,Dallas-Fort Worth-Arlington,Dallas County,134197.0,134246.0,134828.0,135458.0,136178.0,136311.0,135938.0,135741.0,135548.0,136007.0,135947.0,136036.0,136103.0,136474.0,136100.0,135830.0,135445.0,135959.0,137003.0,137943.0,138607.0,138708.0,139321.0,139986.0,139978.0,139698.0,139529.0,139668.0,139828.0,139702.0,139866.0,139817.0,140369.0,140185.0,140342.0,139970.0,140995.0,142293.0,143911.0,145056.0,146050.0,147155.0,148021.0,148752.0,149020.0,149781.0,150354.0,151174.0,151453.0,151981.0,152179.0,152438.0,152507.0,152925.0,153523.0,154229.0,154945.0,155529.0,156210.0,156916.0,157324.0,156706.0,156488.0,156425.0,156938.0,156624.0,156412.0,156468.0,156726.0,157007.0,157193.0,157505.0,157826.0,158494.0,158836.0,159372.0,159813.0,160754.0,161594.0,162218.0,162751.0,163099.0,163440.0,163859.0,163770.0,163593.0,163088.0,163044.0,163334.0,163935.0,164327.0,164794.0,165346.0,166160.0,166713.0,167092.0,168227.0,170550.0,173414.0,175325.0,175713.0,175346.0,175519.0,175902.0,176156.0,176432.0,177212.0,178081.0,179347.0,179024.0,178835.0,178286.0,178909.0,180043.0,180817.0,181506.0,182058.0,181978.0,182140.0,181789.0,181884.0,182804.0,1834

Next we will extract the date columns and transpose them to create a common timeseries accross all RegionID's.

In [4]:
PriceTS = Zdf.loc[:,'1/31/96':].T
PriceTS.rename(index={'RegionID':'Date'})
PriceTS.index = pd.DatetimeIndex(PriceTS.index)
PriceTS.head()

RegionID,274772,112345,192689,270958,118208,270957,192820,192796,268496,194430,112547,198687,111818,267955,272816,273698,272902,199001,32059,267908,194737,275473,272734,403222,25294,195267,275237,273566,35521,343222,274742,267971,273565,193587,194379,275456,28380,10957,273555,270945,273088,275501,193406,193285,276246,276652,403223,273527,275454,276215,276476,269592,403131,343217,343205,118987,275467,193640,276659,273480,274514,271511,41466,275090,270875,273652,41131,33183,18131,15046,27182,267900,34213,113886,47651,275950,46795,54300,274756,115003,268236,48570,403553,27080,40215,196538,403117,270318,343228,37739,116625,118872,18071,273757,273348,343868,48200,276023,40548,275173,232166,271326,271503,403169,270829,275424,197044,273973,403558,13176,274117,238122,343869,273167,246678,275471,274526,270886,273177,273540,45032,274771,403215,403542,193182,276107,273201,268548,26352,403541,124381,276128,13017,276151,193526,274415,13094,112261,269072,273860,403120,275707,343215,276446,54232,267912,274056,274346,117224,403217,269566,48516,274230,30731,267926,269609,118920,229214,244707,269589,273794,273197,343221,276646,275115,273674,343204,269605,111617,5837,193189,343213,343244,274765,274510,271411,403538,115609,268384,214232,31817,114074,274835,118673,199091,270850,269228,267919,37309,275492,117557,27987,403528,275609,273496,32368,271467,51906,189019,276254,343214,269571,273040,276450,46793,275496,268621,275620,46736,154795,273140,195133,117183,193974,114914,123423,138261,154399,247145,269588,403556,272756,274724,275127,57348,214146,275083,275470,403545,275845,37575,6719,274697,116415,271417,268938,270903,343216,403210,343211,271406,274680,195576,273594,272874,403211,276351,118849,49994,273930,276537,271495,274517,269607,271282,269585,274552,269582,403213,271457,273035,270891,269578,273887,26134,274983,271985,241514,403295,273766,276060,28723,753793,274834,47950,113455,269608,154970,268588,274717,271409,276253,155633,275084,270819,268002,268473,343225,47880,343212,194318,7877,28173,274451,37835,275552,25160,6952,113736,220532,343210,46224,268118,116035,276113,46087,273809,275500,276525,274695,343864,403146,154775,269614,268441,396209,403167,6018,194474,271166,115729,403184,272701,274049,275359,273554,192889,117954,250206,403529,270848,417703,121697,39971,272942,403110,271026,276345,403221,21056,417708,274800,152379,245508,403253,215792,272875,113910,155995,269071,9632,343220,271585,204284,236102,116206,196338,396588,270902,275695,208850,270855,117156,273587,115836,403557,762605,114808,273483,275067,763764,202734,44302,240748,274990,275429,271637,403145,259606,403226,403552,272760,272744,403109,155506,38268,268383,118229,121685,193476,270835,269548,403148,111869,236058,342700,275989,403192,274279,275306,753778,270764,763902,763905,118541,271568,753788,343237,267847,268076,270797,271582,274702,274285,269584,273877,8731,214976,269597,274157,271427,763369,274359,403543,269563,753773,8349,214954,274685,276157,154558,344035,232843,268540,274729,140913,761966,268551,27484,274804,271583,343428,275097,203134,270145,21970,403206,275499,276389,152160,275247,274264,268284,17904,270549,268409,271510,270881,275414,268489,270908,268219,272001,5568,117892,267830,275126,140437,38888,761975,207384,270815,272724,268097,274741,251975,125257,273478,270907,272015,195087,191799,268581,23991,276119,15772,192895,126210,15191,403547,269973,269615,273892,55477,118015,272755,116213,269093,243919,271280,4819,273987,762078,276486,206971,276021,275081,23117,268134,274676,417134,55788,271936,764286,403534,268396,19810,26254,273825,276426,273653,274409,33552,259317,49678,623416,761968,272836,403551,275312,273180,113507,275405,761365,275024,270964,275448,275078,269612,271071,275979,753714,276653,268491,271950,275520,273753,195088,763457,403209,764288,272773,403122,210763,275328,276318,36256,274256,276331,753808,200500,271305,271832,274760,273889,403323,275005,273537,753795,235537,271573,268379,275133,403530,18921,272945,271005,155607,274750,269577,116646,121774,33597,

The remaining columns will be preserved in a separate DataFrame 'USneighb', a separate column will be created to aggregate address infromation in order to query a coordinate location object from the GeoPy library.

In [10]:
USneighb = Zdf.loc[:,'SizeRank':'CountyName']
cols = ['RegionName','City','State']
USneighb['Address'] = USneighb[cols].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)
print('Total of {} neighborhoods in the US.'.format(len(USneighb)))
USneighb.head()

Total of 16147 neighborhoods in the US.


,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,Address
RegionID,,,,,,,,,
274772,0,Northeast Dallas,Neighborhood,TX,TX,Dallas,Dallas-Fort Worth-Arlington,Dallas County,"Northeast Dallas, Dallas, TX"
112345,1,Maryvale,Neighborhood,AZ,AZ,Phoenix,Phoenix-Mesa-Scottsdale,Maricopa County,"Maryvale, Phoenix, AZ"
192689,2,Paradise,Neighborhood,NV,NV,Las Vegas,Las Vegas-Henderson-Paradise,Clark County,"Paradise, Las Vegas, NV"
270958,3,Upper West Side,Neighborhood,NY,NY,New York,New York-Newark-Jersey City,New York County,"Upper West Side, New York, NY"
118208,4,South Los Angeles,Neighborhood,CA,CA,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles County,"South Los Angeles, Los Angeles, CA"


### 2.2 Downloading Coordinates data from Geocoder

With the address information aggregated under the 'Address' column we can execute the function for querying coordinates information. For this, a get_coords function was defined, with the goal of bypassing the timeout and recurssion errors that endpoints generate when hiting the service rate quota.

In [106]:
def get_coords(address, attempt=1, max_attempts=5):
    geolocator = Nominatim()
    try:
        # return geolocator.geocode(address)
        location = geolocator.geocode(address,timeout=10000)
        if location == None:
            return float('NaN')
        return [location.latitude, location.longitude]
    except GeocoderTimedOut:
        time.sleep(1)
        if attempt <= max_attempts:
            return do_geocode(address, attempt=attempt+1)
        raise

# USneighb.iloc[10:100,-1].apply(lambda addr: do_geocode(addr))
USneighb['Coords'] = USneighb['Address'].apply(lambda addr: do_geocode(addr))

In [110]:
print('A total of {} out of {} coordinates where found for neighborhoods in the US.'.format(len(USneighb['Coords'].dropna()),len(USneighb)))
USneighb.dropna().head()

A total of 10897 out of 16147 coordinates where found for neighborhoods in the US.


,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,Address,Coords
RegionID,,,,,,,,,,
274772,0,Northeast Dallas,Neighborhood,TX,TX,Dallas,Dallas-Fort Worth-Arlington,Dallas County,"Northeast Dallas, Dallas, TX","(32.7762719, -96.7968559)"
112345,1,Maryvale,Neighborhood,AZ,AZ,Phoenix,Phoenix-Mesa-Scottsdale,Maricopa County,"Maryvale, Phoenix, AZ","(33.4928427, -112.223207)"
192689,2,Paradise,Neighborhood,NV,NV,Las Vegas,Las Vegas-Henderson-Paradise,Clark County,"Paradise, Las Vegas, NV","(36.1672559, -115.1485163)"
270958,3,Upper West Side,Neighborhood,NY,NY,New York,New York-Newark-Jersey City,New York County,"Upper West Side, New York, NY","(40.7870455, -73.9754163)"
118208,4,South Los Angeles,Neighborhood,CA,CA,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles County,"South Los Angeles, Los Angeles, CA","(33.9282906, -118.2788135)"


### 2.3 Individual City Data Validation and Testing

Finally we will inspect the data generated so far for two neighborhoods that will be used to analyze and explore in the second part of this research.

#### 2.3.1 Neighborhoods in each city's data set

In [111]:
LosAngelesNB = USneighb.dropna().loc[(USneighb.City=='Los Angeles') & (USneighb.State=='CA')]
AustinNB = USneighb.dropna().loc[(USneighb.City=='Austin') & (USneighb.State=='TX')]
print('There are {} neighborhoods in LA.'.format(len(LosAngelesNB)))
print('There are {} neighborhoods in Austin.'.format(len(AustinNB)))
LosAngelesNB.head()

There are 93 neighborhoods in LA.
There are 59 neighborhoods in Austin.


,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,Address,Coords
RegionID,,,,,,,,,,
118208,4,South Los Angeles,Neighborhood,CA,CA,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles County,"South Los Angeles, Los Angeles, CA","(33.9282906, -118.2788135)"
268496,8,Southeast Los Angeles,Neighborhood,CA,CA,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles County,"Southeast Los Angeles, Los Angeles, CA","(34.0536909, -118.2427666)"
32059,18,Hollywood,Neighborhood,CA,CA,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles County,"Hollywood, Los Angeles, CA","(34.0980031, -118.3295232)"
274514,62,Mid City,Neighborhood,CA,CA,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles County,"Mid City, Los Angeles, CA","(34.0415271, -118.3603703)"
41466,64,Van Nuys,Neighborhood,CA,CA,Los Angeles,Los Angeles-Long Beach-Anaheim,Los Angeles County,"Van Nuys, Los Angeles, CA","(34.1866193, -118.4486669)"


#### 2.3.1 Here we will extract time window for year 2020 for the neighborhoods contained in the city of Austin. 

### Time Series from Monthly Price Data

In [112]:
PriceTS.loc['2020',AustinNB.index]

RegionID,274685,271391,275057,271652,416968,276540,235693,271395,238833,271529,271286,271398,271588,271570,271311,271296,271651,276459,271572,271497,276473,271353,271574,271422,271667,274993,273989,275255,271665,271437,271303,275649,271587,274900,271400,274730,271339,275246,271613,271325,271500,273924,271474,275969,271632,271361,271551,271393,273354,271576,271360,274045,274474,273440,271653,231200,271327,271442,271611
2020-01-31,324460.0,234571.0,243909.0,399970.0,446682.0,395163.0,285241.0,356310.0,306726.0,352771.0,613887.0,266851.0,292711.0,516894.0,485175.0,699095.0,298997.0,345046.0,375866.0,262317.0,189728.0,513097.0,709128.0,348325.0,755204.0,381011.0,384653.0,422897.0,397320.0,591170.0,719293.0,301545.0,491269.0,799856.0,427878.0,437638.0,483326.0,692579.0,412843.0,476516.0,404977.0,541550.0,405783.0,448785.0,356188.0,334158.0,583397.0,541296.0,268836.0,262643.0,547833.0,484137.0,245346.0,529510.0,864171.0,1392766.0,457932.0,349277.0,499455.0
2020-02-29,327752.0,236131.0,245249.0,401690.0,448048.0,396843.0,288792.0,359164.0,308883.0,354619.0,615386.0,267978.0,291271.0,519155.0,488833.0,701099.0,302092.0,343580.0,378537.0,263669.0,191821.0,514133.0,710012.0,354511.0,758991.0,384150.0,387274.0,423711.0,399334.0,594046.0,719142.0,302370.0,494467.0,801092.0,427844.0,439301.0,485163.0,694876.0,413611.0,477755.0,406131.0,544937.0,404651.0,449167.0,359466.0,336745.0,586577.0,542006.0,269135.0,263017.0,554469.0,487806.0,246400.0,530761.0,864937.0,1393313.0,458780.0,347174.0,502287.0
2020-03-31,330298.0,239320.0,247180.0,404838.0,451228.0,399976.0,292760.0,362780.0,311548.0,357285.0,620657.0,269614.0,291847.0,523943.0,493681.0,706064.0,305666.0,343314.0,382514.0,265252.0,193941.0,517143.0,715328.0,360743.0,766896.0,387426.0,391542.0,427398.0,403062.0,600788.0,724000.0,304295.0,500241.0,808338.0,430493.0,442535.0,488624.0,700402.0,416845.0,480464.0,408837.0,550003.0,405623.0,452946.0,363024.0,340587.0,593391.0,547229.0,270585.0,264335.0,559511.0,492708.0,247921.0,536849.0,874256.0,1402283.0,461540.0,347606.0,509305.0
2020-04-30,333381.0,243145.0,248831.0,408982.0,455102.0,403255.0,297225.0,366725.0,314323.0,360945.0,624699.0,270562.0,292865.0,529731.0,498503.0,711540.0,308979.0,344334.0,387251.0,266385.0,196770.0,521281.0,721070.0,369630.0,774906.0,392489.0,396377.0,431571.0,407128.0,609868.0,730527.0,306109.0,504176.0,816921.0,433005.0,445452.0,491516.0,706043.0,420015.0,483485.0,412255.0,556503.0,408275.0,458603.0,367011.0,344082.0,600543.0,553388.0,271838.0,266088.0,566056.0,498105.0,249755.0,544392.0,881171.0,1409187.0,464877.0,349012.0,518207.0


## 3.0 Data Exploration

### 3.1 Explore and cluster the neighborhoods in Los Angeles

Since latitude and Longitude infromation was captured for Postal Codes which are matched 1 to 1 with more than one neighborhood, we will focus on exploring those instead of neighborhoods.

Foursquare Venues Data Download Function Definition: 

In [135]:
# CLIENT_ID = '' #your Foursquare ID
# CLIENT_SECRET = '' #your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 1000 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            LIMIT, # limit of number of venues returned by Foursquare API
            radius # define radius
            )  
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
   
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Executing Function for finding venue infromation by each Toronto Borough.

In [167]:
Austin_venues = getNearbyVenues(names=AustinNB['RegionName'],
                                   latitudes=AustinNB['Coords'].str[0],
                                   longitudes=AustinNB['Coords'].str[1]
                                  )
print(Austin_venues.shape)
Austin_venues.head()

(2852, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,North Austin,30.40233,-97.726009,Fleming's Prime Steakhouse & Wine Bar,30.403173,-97.725581,Steakhouse
1,North Austin,30.40233,-97.726009,The Domain,30.400602,-97.725121,Shopping Plaza
2,North Austin,30.40233,-97.726009,Tiffany & Co.,30.403335,-97.724586,Jewelry Store
3,North Austin,30.40233,-97.726009,Louis Vuitton,30.403620,-97.724931,Boutique
4,North Austin,30.40233,-97.726009,North Italia,30.402334,-97.725875,Italian Restaurant


In [169]:
LosAngeles_venues = getNearbyVenues(names=LosAngelesNB['RegionName'],
                                   latitudes=LosAngelesNB['Coords'].str[0],
                                   longitudes=LosAngelesNB['Coords'].str[1]
                                  )
print(LosAngeles_venues.shape)
LosAngeles_venues.head()

(4925, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,South Los Angeles,33.928291,-118.278813,Imperial Fish Market,33.931344,-118.278068,Seafood Restaurant
1,South Los Angeles,33.928291,-118.278813,Jack in the Box,33.930720,-118.282199,Fast Food Restaurant
2,South Los Angeles,33.928291,-118.278813,Wingstop,33.931526,-118.283082,Wings Joint
3,South Los Angeles,33.928291,-118.278813,McDonald's,33.931276,-118.282927,Fast Food Restaurant
4,South Los Angeles,33.928291,-118.278813,SUBWAY,33.930470,-118.287409,Sandwich Place


A total of 4,922 venues were found. Let's aggregate them by categories and match such categories by borough.

In [59]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
# one hot encoding
toronto_venues_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_venues_onehot['Borough'] = toronto_venues['Borough']

toronto_venues_grouped = toronto_venues_onehot.groupby('Borough').mean().reset_index()
toronto_venues_grouped.head()

There are 328 uniques categories.


,Borough,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundry Service,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,South American Restaurant,

It would be interesting to explore what are the most common type of venues by borough. Next we will define a function for sorting venues by frequency and create a new dataframe for boroughts with corresponding 10 most common venue categories.

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [61]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = toronto_venues_grouped['Borough']

for ind in np.arange(toronto_venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Restaurant,Park,Gym,Pizza Place,Pharmacy,Thai Restaurant
1,Downtown Toronto,Coffee Shop,Café,Japanese Restaurant,Restaurant,Park,Hotel,Gastropub,Theater,Bakery,Italian Restaurant
2,East Toronto,Coffee Shop,Café,Park,Greek Restaurant,Pizza Place,Pub,Bakery,Brewery,Bar,Indian Restaurant
3,East York,Coffee Shop,Pizza Place,Café,Grocery Store,Greek Restaurant,Sandwich Place,Bank,Fast Food Restaurant,Park,Pharmacy
4,Etobicoke,Park,Pizza Place,Coffee Shop,Grocery Store,Pharmacy,Restaurant,Bank,Italian Restaurant,Sandwich Place,Discount Store


### 3.2 Clustering Boroughs into groups by venue types charactersitics.

Define k=10 clusters for the total of 10 boroughs we collected venue infromation for.

In [62]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering = toronto_venues_grouped.drop(['Borough'],axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([6, 2, 7, 0, 4, 1, 9, 5, 8, 3], dtype=int32)

In [63]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = LatLongNeigh

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Borough'), on='Borough')

toronto_merged.head(10) # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,9,Coffee Shop,Park,Pizza Place,Restaurant,Bank,Grocery Store,Japanese Restaurant,Fast Food Restaurant,Sandwich Place,Pharmacy
1,M4A,North York,Victoria Village,43.725882,-79.315572,9,Coffee Shop,Park,Pizza Place,Restaurant,Bank,Grocery Store,Japanese Restaurant,Fast Food Restaurant,Sandwich Place,Pharmacy
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Café,Japanese Restaurant,Restaurant,Park,Hotel,Gastropub,Theater,Bakery,Italian Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,9,Coffee Shop,Park,Pizza Place,Restaurant,Bank,Grocery Store,Japanese Restaurant,Fast Food Restaurant,Sandwich Place,Pharmacy
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Café,Japanese Restaurant,Restaurant,Park,Hotel,Gastropub,Theater,Bakery,Italian Restaurant
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,4,Park,Pizza Place,Coffee Shop,Grocery Store,Pharmacy,Restaurant,Bank,Italian Restaurant,Sandwich Place,Discount Store
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,5,Chinese Restaurant,Coffee Shop,Pizza Place,Fast Food Restaurant,Park,Bakery,Bank,Pharmacy,Restaurant,Grocery Store
7,M3B,North York,Don Mills,43.745906,-79.352188,9,Coffee Shop,Park,Pizza Place,Restaurant,Bank,Grocery Store,Japanese Restaurant,Fast Food Restaurant,Sandwich Place,Pharmacy
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,0,Coffee Shop,Pizza Place,Café,Grocery Store,Greek Restaurant,Sandwich Place,Bank,Fast Food Restaurant,Park,Pharmacy
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Coffee Shop,Café,Japanese Restaurant,Restaurant,Park,Hotel,Gastropub,Theater,Bakery,Italian Restaurant


It is interesting to observe that while the borough name was not included as a feature to run kmeans against. The Borough os North York always fell under classification group 9. Showing that the classfiication algorithm was able to deduce the borough based only on the venues type on each borough.

### 3.3 Maps use for visualizing neighborhoods and how they cluster together 

In [97]:
address = Addressdf.loc[118208,'combined']
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of _ are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of _ are 33.9282906, -118.2788135.


Finally we will display a map with the different classified groups color coded. It is evident that some correlation exists between geographical classification and venue-based classification. 

In [57]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Borough'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters